In [1]:
import numpy as np
import datajoint as dj

## Connect to DB

In [2]:
dj.config['database.host'] = '127.0.0.1'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
schema = dj.schema('tutorial', locals())

Connecting root@127.0.0.1:3306


## Define and create table

In [3]:
@schema
class Mouse(dj.Manual):
      definition = """
      # mouse
      mouse_id: int                  # unique mouse id
      ---
      dob: date                      # mouse date of birth
      sex: enum('M', 'F', 'U')    # sex of mouse - Male, Female, or Unknown/Unclassified
      """

In [4]:
mouse = Mouse()
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"


## Populate table

In [5]:
mouse.insert1((0, '2017-03-01', 'M') )

In [6]:
data = {
  'mouse_id': 100,
  'dob': '2017-05-12',
  'sex': 'F'
}

In [7]:
mouse.insert1(data)

In [8]:
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
100,2017-05-12,F


In [9]:
data = [
  (1, '2016-11-19', 'M'),
  (2, '2016-11-20', 'U'),
  (5, '2016-12-25', 'F')
]

# now insert all at once
mouse.insert(data)

In [10]:
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
1,2016-11-19,M
2,2016-11-20,U
5,2016-12-25,F
100,2017-05-12,F


In [11]:
# tuples or dicts possible
data = [
  {'mouse_id': 10, 'dob': '2017-01-01', 'sex': 'F'},
  {'mouse_id': 11, 'dob': '2017-01-03', 'sex': 'F'},
]

# insert them all
mouse.insert(data)

In [12]:
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
1,2016-11-19,M
2,2016-11-20,U
5,2016-12-25,F
10,2017-01-01,F
11,2017-01-03,F
100,2017-05-12,F


## Persist changes
place table definitions in a file and import

In [5]:
# import os
# from src.tutorial_tables import *


## Querying the table 

In [15]:
data = mouse.fetch()

In [16]:
type(data)

numpy.ndarray

In [17]:
data

array([(  0, datetime.date(2017, 3, 1), 'M'),
       (  1, datetime.date(2016, 11, 19), 'M'),
       (  2, datetime.date(2016, 11, 20), 'U'),
       (  5, datetime.date(2016, 12, 25), 'F'),
       ( 10, datetime.date(2017, 1, 1), 'F'),
       ( 11, datetime.date(2017, 1, 3), 'F'),
       (100, datetime.date(2017, 5, 12), 'F')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex', 'O')])

In [18]:
data_dict = mouse.fetch(as_dict=True)

In [19]:
type(data_dict)

list

In [20]:
data_dict

[{'mouse_id': 0, 'dob': datetime.date(2017, 3, 1), 'sex': 'M'},
 {'mouse_id': 1, 'dob': datetime.date(2016, 11, 19), 'sex': 'M'},
 {'mouse_id': 2, 'dob': datetime.date(2016, 11, 20), 'sex': 'U'},
 {'mouse_id': 5, 'dob': datetime.date(2016, 12, 25), 'sex': 'F'},
 {'mouse_id': 10, 'dob': datetime.date(2017, 1, 1), 'sex': 'F'},
 {'mouse_id': 11, 'dob': datetime.date(2017, 1, 3), 'sex': 'F'},
 {'mouse_id': 100, 'dob': datetime.date(2017, 5, 12), 'sex': 'F'}]

In [21]:
sex, ids = mouse.fetch('sex', 'mouse_id')

In [22]:
sex

array(['M', 'M', 'U', 'F', 'F', 'F', 'F'], dtype=object)

In [23]:
ids

array([  0,   1,   2,   5,  10,  11, 100])

In [24]:
mouse & 'mouse_id = 0'

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M


In [25]:
mouse & 'sex = "M"'

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
1,2016-11-19,M


In [26]:
mouse & 'dob > "2017-01-01"' & 'sex = "M"'

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M


In [27]:
type(mouse & 'dob > "2017-01-01"' & 'sex = "M"')

__main__.Mouse

In [28]:
# using a dict for the query
r = {
      'sex': 'M',
      'mouse_id' : 0
    }
mouse & r

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M


In [29]:
# collect final result
(mouse & r).fetch()

array([(0, datetime.date(2017, 3, 1), 'M')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex', 'O')])

In [30]:
# delete rows in the db
(mouse & 'mouse_id = 10').delete()

About to delete:
`tutorial`.`mouse`: 1 items
Proceed? [yes, No]: no
Cancelled deletes.


## Child tables

In [4]:
@schema
class Session(dj.Manual):
    definition = """
    # experiment session
    -> Mouse
    session_date: date            # session date
    ---
    experiment_setup: int         # experiment setup ID
    experimenter: varchar(128)    # name of the experimenter
    """


In [5]:
# added session table definition in src/tutorials_tables.py
# -> indicates primary key dependency
session = Session()

In [6]:
session

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Walker
0,2017-05-19,0,Edgar Walker
5,2017-01-05,1,Fabian Sinz
100,2017-05-25,1,Jake Reimer


In [33]:
data = {
      'mouse_id': 0,
      'session_date': '2017-05-15',
      'experiment_setup': 0,
      'experimenter': 'Edgar Y. Walker'
    }
session.insert1(data)

In [34]:
session

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Y. Walker


In [35]:
# with unexisting mouse_id
data = {
      'mouse_id': 12,
      'session_date': '2017-05-15',
      'experiment_setup': 0,
      'experimenter': 'Edgar Y. Walker'
    }
session.insert1(data)

IntegrityError: Cannot add or update a child row: a foreign key constraint fails (`tutorial`.`session`, CONSTRAINT `session_ibfk_1` FOREIGN KEY (`mouse_id`) REFERENCES `mouse` (`mouse_id`) ON UPDATE CASCADE)

In [36]:
# Deletion with foreign key contraints
(mouse & 'mouse_id = 0').delete() # cascading delete happens

About to delete:
`tutorial`.`session`: 1 items
`tutorial`.`mouse`: 1 items
Proceed? [yes, No]: no
Cancelled deletes.


## Queries with multiple tables

In [37]:
session

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Y. Walker


In [17]:
session.insert([('0','2018-05-01', 1, 'maj'), ('100','2019-07-08', 1, 'maj')])

In [18]:
session

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Y. Walker
0,2018-05-01,1,maj
100,2019-07-08,1,maj


In [19]:
mouse & session

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
100,2017-05-12,F


In [20]:
session & mouse # should be same as session table

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Y. Walker
0,2018-05-01,1,maj
100,2019-07-08,1,maj


In [21]:
session & (mouse & 'sex = "M"') # sessions for which the mouse is male

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Y. Walker
0,2018-05-01,1,maj


In [23]:
mouse & (session & "session_date >= 2017-05-19") # mice who had session done after 19 may 2017

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
100,2017-05-12,F


In [24]:
# difference operators
mouse - session # all mice that don't have any sessions

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
1,2016-11-19,M
2,2016-11-20,U
5,2016-12-25,F
10,2017-01-01,F
11,2017-01-03,F


In [25]:
session - mouse # should be empty

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter


In [26]:
mouse * session # inner join

mouse_id unique mouse id,session_date session date,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified",experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,2017-03-01,M,0,Edgar Y. Walker
0,2018-05-01,2017-03-01,M,1,maj
100,2019-07-08,2017-05-12,F,1,maj


In [27]:
session * mouse # same, different column order

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-05-15,0,Edgar Y. Walker,2017-03-01,M
0,2018-05-01,1,maj,2017-03-01,M
100,2019-07-08,1,maj,2017-05-12,F


## Importing data

In [8]:
import os
os.getcwd()

'/home/ahmedj/onboarding-mathis/notebooks'

In [9]:
session

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Walker
0,2017-05-19,0,Edgar Walker
5,2017-01-05,1,Fabian Sinz
100,2017-05-25,1,Jake Reimer


In [54]:
session.delete()

About to delete:
`tutorial`.`session`: 3 items
Proceed? [yes, No]: yes
Committed.


In [55]:
session

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter


In [56]:
sessions = [(0,'2017-05-15', 0, "Edgar Walker"), (0,'2017-05-19', 0, "Edgar Walker"),
            (5,'2017-01-05', 1, "Fabian Sinz"),  (100,'2017-05-25', 1, "Jake Reimer")] # copied from tutorial

In [57]:
session.insert(sessions)

In [58]:
session

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Walker
0,2017-05-19,0,Edgar Walker
5,2017-01-05,1,Fabian Sinz
100,2017-05-25,1,Jake Reimer


In [61]:
data = np.load('../data/data_100_2017-05-25.npy') # downloaded from tutorial

In [63]:
data.shape

(1000,)

In [78]:
Session()

mouse_id unique mouse id,session_date session date,experiment_setup experiment setup ID,experimenter name of the experimenter
0,2017-05-15,0,Edgar Walker
0,2017-05-19,0,Edgar Walker
5,2017-01-05,1,Fabian Sinz
100,2017-05-25,1,Jake Reimer


In [5]:
# dj.Imported instead of dj.Manual
@schema
class Neuron(dj.Imported):
    definition = """
    -> Session
    ---
    activity:  longblob    # electric activity of the neuron
    """
    def _make_tuples(self, key):    
        file_path = f"../data/data_{key['mouse_id']}_{key['session_date']}.npy"
        data = np.load(file_path)
        key['activity'] = data
        self.insert1(key)
        

In [32]:
neuron = Neuron()
neuron

mouse_id unique mouse id,session_date session date,activity longblob used to store array


In [33]:
neuron.parents()

['`tutorial`.`session`']

In [34]:
session.parents()

['`tutorial`.`mouse`']

In [35]:
neuron.populate()

In [36]:
neuron

mouse_id unique mouse id,session_date session date,activity longblob used to store array
0,2017-05-15,=BLOB=
0,2017-05-19,=BLOB=
5,2017-01-05,=BLOB=
100,2017-05-25,=BLOB=


In [37]:
neuron.populate()

In [39]:
neuron # nothing happens, no erroes

mouse_id unique mouse id,session_date session date,activity longblob used to store array
0,2017-05-15,=BLOB=
0,2017-05-19,=BLOB=
5,2017-01-05,=BLOB=
100,2017-05-25,=BLOB=


## Computed tables

In [6]:
# dj.computed, operations on an existing table

@schema
class ActivityStatistics(dj.Computed):
    definition = """
    -> Neuron
    ---
    mean: float    # mean activity
    stdev: float   # standard deviation of activity
    max: float     # maximum activity
    """

    def _make_tuples(self, key):
        activity = (Neuron() & key).fetch1('activity')    # fetch1 gives the attribute not in list
        key['mean'] = activity.mean()   
        key['stdev'] = activity.std()   
        key['max'] = activity.max()     
        self.insert1(key)


In [7]:
stats = ActivityStatistics()

In [8]:
stats

mouse_id unique mouse id,session_date session date,mean mean activity,stdev standard deviation of activity,max maximum activity


In [10]:
Neuron()

mouse_id unique mouse id,session_date session date,activity longblob used to store array
0,2017-05-15,=BLOB=
0,2017-05-19,=BLOB=
5,2017-01-05,=BLOB=
100,2017-05-25,=BLOB=


In [11]:
stats.parents()

['`tutorial`.`_neuron`']

In [12]:
stats.populate()

In [13]:
stats

mouse_id unique mouse id,session_date session date,mean mean activity,stdev standard deviation of activity,max maximum activity
0,2017-05-15,0.207357,0.400867,2.48161
0,2017-05-19,0.13274,0.291462,1.82805
5,2017-01-05,0.0891786,0.236412,1.37389
100,2017-05-25,0.21907,0.328783,1.76383


## Lookup tables

In [7]:
# Threshold table for spike detection
@schema
class SpikeDetectionParam(dj.Lookup):
    definition = """
    sdp_id: int      # unique id for spike detection parameter set
    ---
    threshold: float   # threshold for spike detection
    """

In [18]:
@schema 
class Spikes(dj.Computed):
    definition = """
    -> Neuron
    -> SpikeDetectionParam
    ---
    spikes: longblob  
    count: int
    """
    # make_tuples will be called for every combination of neuron x threshold
    def _make_tuples(self, key):
        activity = (Neuron() & key).fetch1('activity') # even if key has extra items, they are ignored
        threshold = (SpikeDetectionParam() & key).fetch1('threshold') # even if key has extra items, they are ignored
        
        spikes = np.zeros(activity.shape)
        spikes[activity > threshold] = 1
        key['spikes'] = spikes
        key['count'] = spikes.sum()
        
        self.insert1(key)

In [19]:
spikes = Spikes()

In [20]:
spikes

mouse_id unique mouse id,session_date session date,sdp_id unique id for spike detection parameter set,spikes,count


In [21]:
spikes.populate()

In [14]:
sdp = SpikeDetectionParam()
sdp

sdp_id unique id for spike detection parameter set,threshold threshold for spike detection


In [15]:
sdp.insert([(1,0.9), (2,1.3), (3, 0.5)])

In [16]:
sdp

sdp_id unique id for spike detection parameter set,threshold threshold for spike detection
1,0.9
2,1.3
3,0.5


In [22]:
spikes 

mouse_id unique mouse id,session_date session date,sdp_id unique id for spike detection parameter set,spikes,count
0,2017-05-15,1,=BLOB=,82
0,2017-05-15,2,=BLOB=,31
0,2017-05-15,3,=BLOB=,150
0,2017-05-19,1,=BLOB=,36
0,2017-05-19,2,=BLOB=,8
0,2017-05-19,3,=BLOB=,106
5,2017-01-05,1,=BLOB=,20
5,2017-01-05,2,=BLOB=,1
5,2017-01-05,3,=BLOB=,73
100,2017-05-25,1,=BLOB=,54


In [23]:
spikes * sdp # together with the the threshold value

mouse_id unique mouse id,session_date session date,sdp_id unique id for spike detection parameter set,spikes,count,threshold threshold for spike detection
0,2017-05-15,1,=BLOB=,82,0.9
0,2017-05-19,1,=BLOB=,36,0.9
5,2017-01-05,1,=BLOB=,20,0.9
100,2017-05-25,1,=BLOB=,54,0.9
0,2017-05-15,2,=BLOB=,31,1.3
0,2017-05-19,2,=BLOB=,8,1.3
5,2017-01-05,2,=BLOB=,1,1.3
100,2017-05-25,2,=BLOB=,12,1.3
0,2017-05-15,3,=BLOB=,150,0.5
0,2017-05-19,3,=BLOB=,106,0.5


## Altering the table

In [36]:
# define a dummy table
@schema
class Dummy(dj.Manual):
    definition = """
    id: int
    ---
    field_a: float
    field_b: varchar(10)
    field_c: int
    """

In [37]:
# add some rows

dummy = Dummy()
data = [(0, 0.1, 'abc', 2),
        (1, 0.2, 'efg', 2),
        (2, 0.9, 'hij', 3)]
dummy.insert(data)

In [38]:
dummy

id,field_a,field_b,field_c
0,0.1,abc,2
1,0.2,efg,2
2,0.9,hij,3


In [39]:
# one extra field added to definition
@schema
class Dummy(dj.Manual):
    definition = """
    id: int
    ---
    field_a: float
    field_b: varchar(10)
    field_c: int
    field_d: int
    """

In [42]:
dummy = Dummy()
print(dummy.definition)
dummy # table not changed


    id: int
    ---
    field_a: float
    field_b: varchar(10)
    field_c: int
    field_d: int
    


id,field_a,field_b,field_c
0,0.1,abc,2
1,0.2,efg,2
2,0.9,hij,3


In [43]:
dummy.alter()

ALTER TABLE `tutorial`.`dummy`
	ADD `field_d` int NOT NULL AFTER `field_c`

Execute? [yes, no]: yes
Table altered


In [44]:
dummy # need to reistantiate

id,field_a,field_b,field_c
0,0.1,abc,2
1,0.2,efg,2
2,0.9,hij,3


In [45]:
dummy = Dummy()
dummy # default values added, can define our own default values

id,field_a,field_b,field_c,field_d
0,0.1,abc,2,0
1,0.2,efg,2,0
2,0.9,hij,3,0


In [47]:
dummy.insert1((3, 1, 'klm', 1, 9))
dummy

id,field_a,field_b,field_c,field_d
0,0.1,abc,2,0
1,0.2,efg,2,0
2,0.9,hij,3,0
3,1.0,klm,1,9


In [48]:
# does not work for primary and foreign keys
@schema
class Dummy(dj.Manual):
    definition = """
    id: int
    id_new: int
    ---
    field_a: float
    field_b: varchar(10)
    field_c: int
    field_d: int
    """

In [49]:
dummy = Dummy()
dummy.alter()

NotImplementedError: table.alter cannot alter the primary key (yet).